In [58]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models

In [22]:
# Set the path to the directory containing audio files and annotations
data_dir = "./applications/data/TUT-sound-events-2017-development"

In [23]:
import csv

# Open input and output files
with open('./applications/data/TUT-sound-events-2017-development/meta.txt', 'r') as f_in, open('output.csv', 'w', newline='') as f_out:
    # Create CSV writer
    writer = csv.writer(f_out)

    # Write header row
    writer.writerow(['filename', 'events', 'event_type'])

    # Loop over lines in input file
    for line in f_in:
        # Split line into components
        line_data = line.strip().split('\t')
        filename=line_data[0]
        start_time = line_data[2] 
        end_time = line_data[3]
        event_type = line_data[4] + line_data[5]
        event_type = event_type.replace('mixture','')
        # Format events string
        events_str = '{},{}'.format(float(start_time) * 1000, float(end_time) * 1000)

        # Write row to output file
        writer.writerow([filename, events_str, event_type])

In [24]:
# Load data from annotations file
annotations_file = os.path.join("output.csv")
annotations = pd.read_csv(annotations_file)

In [59]:
def extract_features(file_path):
    signal, sr = librosa.load(file_path, sr=44100)
    mfccs = librosa.feature.mfcc(signal, sr=sr, n_mfcc=20)
    return np.mean(mfccs.T,axis=0)

In [60]:
# Preprocess data
X = []
y = []
for index, row in annotations.iterrows():
    file_path = os.path.join(data_dir, row['filename'])
    features = extract_features(file_path)
    X.append(features)
    y.append(row['event_type'])

c:\Users\namit\miniconda3\envs\tf2-dcase\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[0.01743829 0.01798326 0.01816159 ... 0.0016017  0.00169945 0.00179231] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\namit\miniconda3\envs\tf2-dcase\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[ 0.00354946  0.0035364   0.00356048 ... -0.00657278 -0.00671083
 -0.00672311] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\namit\miniconda3\envs\tf2-dcase\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Pass y=[-0.00764149 -0.00754553 -0.00747216 ... -0.00069636 -0.00050914
 -0.00034547] as keyword args. From version 0.10 passing these as positional arguments will result in an err

KeyboardInterrupt: 

In [57]:
# Convert labels to numeric values
label_map = {label: i for i, label in enumerate(set(y))}
y = [label_map[label] for label in y]

TypeError: Required argument 'object' (pos 1) not found

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build model architecture
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train[0].shape)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(label_map), activation='softmax'))

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32)


In [ ]:
# Evaluate model on test set
y_pred = model.predict_classes(np.array(X_test))
f1 = f1_score(y_test, y_pred, average='weighted')
acc = accuracy_score(y_test, y_pred)

In [ ]:
print('F1 score:', f1)
print('Accuracy:', acc)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN model
inputs = Input(shape=X_train[0].shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2)(x)
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# Compile and train model
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

# Predict on testing data and calculate metrics
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print("F1 score: ", f1)
print("Accuracy: ", accuracy)

In [ ]:
""" for event in events:
        event_start = event[0]
        event_end = event[1]
        y.append([event_start, event_end])
        X.append(row["filename"]) """